# Building Machine Learning Classifiers: Random Forest on a holdout test set

### Read in & clean text

In [436]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, vstack
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("./SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

print(data.iloc[0]['body_text'])

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


body_len  punct%    0    1    2    3    4    5    6    7  ...  8094  8095  \
0       128     4.7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1        49     4.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2        62     3.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3        28     7.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4       135     4.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   

   8096  8097  8098  8099  8100  8101  8102  8103  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 8106 columns]

### Explore RandomForestClassifier through Holdout Set

In [437]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [438]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [439]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [440]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse = True)[0:10]

[(0.033873418146601525, 'body_len'),
 (0.030478146197332003, 3134),
 (0.026559388816223703, 4796),
 (0.026272797845580134, 2031),
 (0.02490251359978206, 6285),
 (0.024067838201064052, 7350),
 (0.021826292612080214, 5724),
 (0.021755344591219673, 7027),
 (0.020923985053007264, 1803),
 (0.01637209453899703, 1361)]

In [441]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [442]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.606 / Accuracy: 0.945


In [435]:
custom_message = "How are the kids Jack? I saw them at the soccer game LOL"

In [434]:
def predict_spam(custom_message):
    data_copy = pd.read_csv("./SMSSpamCollection.tsv", sep='\t')
    data_copy.columns = ['label', 'body_text']

    data_copy.loc[-1] = ['ham', custom_message]
    data_copy.index = data_copy.index + 1  
    data_copy = data_copy.sort_index()
    
    # shifting index
    
    data_copy['body_len'] = data_copy['body_text'].apply(lambda x: len(x) - x.count(" "))
    data_copy['punct%'] = data_copy['body_text'].apply(lambda x: count_punct(x))

    custom_tfidf = tfidf_vect.fit_transform(data_copy['body_text'])
    features = pd.concat([data_copy['body_len'], data_copy['punct%'], pd.DataFrame(custom_tfidf.toarray())], axis=1)
    
    prediction = rf_model.predict(features.iloc[0:1])[0]
    return prediction

ValueError: Number of features of the model must match the input. Model n_features is 8106 and input n_features is 8107 